In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Online prediction with BigQuery ML

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/bqml-online-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/bqml-online-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/bigquery_ml/bqml-online-prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook is aimed at data analysts and data scientists who have data in BigQuery, want to train a model using BigQuery ML, register the model to Vertex AI Model Registry, and deploy it to an endpoint for real-time prediction. 

Learn more about [BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml).

### Objective

In this tutorial, you learn how to train and deploy a churn prediction model for real-time inference, with the data in BigQuery and model trained using BigQuery ML, registered to Vertex AI Model Registry, and deployed to an endpoint on Vertex AI for online predictions.

This tutorial uses the following Google Cloud data analytics and ML services:

- BigQuery
- BigQuery ML
- Vertex AI Model Registry
- Vertex endpoints


The steps performed include:

- Using Python & SQL to query the public data in BigQuery
- Preparing the data for modeling
- Training a classification model using BigQuery ML and registering it to Vertex AI Model Registry
- Inspecting the model on Vertex AI Model Registry
- Deploying the model to an endpoint on Vertex AI
- Making sample online predictions to the model endpoint


### Dataset

The dataset, <a href="https://console.cloud.google.com/bigquery?project=bigquery-public-data&d=ga4_obfuscated_sample_ecommerce&p=bigquery-public-data&page=dataset" target="_blank">available publicly on BigQuery</a>, comes from obfuscated <a href="https://support.google.com/analytics/answer/10937659" target="_blank">Google Analytics 4 data</a> from the <a href="https://shop.googlemerchandisestore.com/" target="_blank">Google Merchandise Store</a>).

### Costs 

This tutorial uses billable components of Google Cloud:

* BigQuery
* BigQuery ML
* Vertex AI


Learn about <a href="https://cloud.google.com/bigquery/pricing" target="_blank">BigQuery Pricing</a>, <a href="https://cloud.google.com/bigquery-ml/pricing" target="_blank">BigQuery ML pricing</a>, <a href="https://cloud.google.com/vertex-ai/pricing" target="_blank">Vertex AI
pricing</a>, and use the <a href="https://cloud.google.com/products/calculator/" target="_blank">Pricing
Calculator</a>
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to <a href="https://cloud.google.com/python/setup" target="_blank">Setting up a Python development
environment</a> and the <a href="https://jupyter.org/install" target="_blank">Jupyter
installation guide</a> provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. <a href="https://cloud.google.com/sdk/docs/" target="_blank">Install and initialize the Cloud SDK.</a>

1. <a href="https://cloud.google.com/python/setup#installing_python" target="_blank">Install Python 3.</a>

1. <a href="https://cloud.google.com/python/setup#installing_and_using_virtualenv" target="_blank">Install
   virtualenv</a>
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install the following packages required to execute this notebook. 

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q google-cloud-bigquery db-dtypes

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [1]:
# # Automatically restart kernel after installs
# import os

# if not os.getenv("IS_TESTING"):
#     # Automatically restart kernel after installs
#     import IPython

#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. <a href="https://console.cloud.google.com/cloud-resource-manager" target="_blank">Select or create a Google Cloud project</a>. When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. <a href="https://cloud.google.com/billing/docs/how-to/modify-project" target="_blank">Make sure that billing is enabled for your project</a>.

1. <a href="https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com" target="_blank">Enable the Vertex AI API</a>.

1. If you are running this notebook locally, you will need to install the <a href="https://cloud.google.com/sdk" target="_blank">Cloud SDK</a>.

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [2]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [3]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: mwpmltr


Otherwise, set your project ID here.

In [4]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You might not be able to use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/general/locations" target="_blank">Vertex AI regions</a>.

In [5]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
# import random
# import string


# # Generate a uuid of a specifed length(default=8)
# def generate_uuid(length: int = 8) -> str:
#     return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


# UUID = generate_uuid()

In [6]:
UUID='0obvksai'

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the <a href="https://console.cloud.google.com/apis/credentials/serviceaccountkey" target="_blank">**Create service account key** page</a>.

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [7]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [8]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [9]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 55590906972-compute@developer.gserviceaccount.com


### Import libraries

In [10]:
from typing import Union

import google.cloud.aiplatform as vertex_ai
import pandas as pd
from google.cloud import bigquery

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [11]:
vertex_ai.init(project=PROJECT_ID, location=REGION)

Create the BigQuery client.

In [22]:
bq_client = bigquery.Client(project=PROJECT_ID, location=REGION)

Use a helper function for sending queries to BigQuery.

In [23]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

## BigQuery ML introduction

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification, regression, forecasting, and matrix factorization, in BigQuery using SQL syntax directly. BigQuery ML uses the scalable infrastructure of BigQuery ML so you don't need to set up additional infrastructure for training or batch serving.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs" target="_blank">BigQuery ML documentation</a>.

In [24]:
BQ_DATASET_NAME = f"ga4_churnprediction_{UUID}"

sql_create_dataset = f"""CREATE SCHEMA IF NOT EXISTS {BQ_DATASET_NAME}"""

print(sql_create_dataset)

run_bq_query(sql_create_dataset)

CREATE SCHEMA IF NOT EXISTS ga4_churnprediction_0obvksai
Finished job_id: 0c4ed0a8-a3dc-4a7b-a524-e320fa2ddfeb


""


### Inspect the pre-processed Google Analytics 4 data

Inpect data that has been pre-processed from <a href="https://support.google.com/analytics/answer/10937659" target="_blank">Google Analytics 4 data from the Google Merchandise Store</a> so that it can be used for classification. For more information on how this data was prepared, read <a href="https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml" target="_blank">this blog post</a>.

As seen below, each row represents a single user, and the columns represent their demographic features, their aggregated behavioral features in the first 24 hours of visiting the Google Merchandise Store, and the label (whether the user churned or returned any time after the first 24 hours).

In [26]:
bq_client = bigquery.Client(project=PROJECT_ID, location='US')

In [27]:
sql_inspect = """
SELECT
    *
FROM
    `bqmlpublic.demo_ga4churnprediction.training_data`
LIMIT
    100
"""
run_bq_query(sql_inspect)

Finished job_id: 1745f1f2-fac2-4f72-b487-dfa1dfbb31a6


,user_pseudo_id,country,operating_system,language,cnt_user_engagement,cnt_page_view,cnt_view_item,cnt_view_promotion,cnt_select_promotion,cnt_add_to_cart,cnt_begin_checkout,cnt_add_shipping_info,cnt_add_payment_info,cnt_purchase,user_first_engagement,month,julianday,dayofweek,churned
0,5298013.7728745800,United States,Web,en-us,5,7,0,1,0,0,0,0,0,0,2021-01-01 13:59:53.752059+00:00,1,1,6,0
1,22288916.8827658539,United Kingdom,Web,None,11,13,4,3,1,3,0,0,0,0,2021-01-01 11:21:20.741223+00:00,1,1,6,0
2,3573143.3095259411,United States,Web,en-us,2,3,0,0,0,0,0,0,0,0,2021-01-01 06:23:08.482574+00:00,1,1,6,1
3,5115768.1503494300,United States,Web,en-us,1,2,0,0,0,0,0,0,0,0,2021-01-01 20:07:22.264757+00:00,1,1,6,0
4,2914301.0348640426,Austria,Windows,en-us,6,8,0,3,0,0,0,0,0,0,2021-01-01 23:30:06.990337+00:00,1,1,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,13623777.5841147931,United States,Web,en-us,8,10,0,1,0,0,0,0,0,0,2021-01-01 01:16:07.770257+00:00,1,1,6,0
96,6351413.2459505432,Morocco,Web,en-us,9,11,5,0,0,0,0,0,0,0,2021-01-01 01:00:01.910471+00:00,1,1,6,1
97,8284309.3576851305,United Kingdom,Web,en-gb,3,5,0,2,0,0,0,0,0,0,2021-01-01 19:10:08.402348+00:00,1,1,6,0
98,44673777.8459608727,United States,Web,None,1,2,0,0,0,0,0,0,0,0,2021-01-01 10:13:22.966736+00:00,1,1,6,0


### Train a classification model using BigQuery ML

The query below trains a logistic regression model using BigQuery ML. BigQuery resources are used to train the model.

In the `OPTIONS` parameter:
* with `model_registry="vertex_ai"`, the BigQuery ML model will automatically be <a href="https://cloud.google.com/vertex-ai/docs/model-registry/model-registry-bqml" target="_blank">registered to Vertex AI Model Registry</a>, which enables you to view all of your registered models and its versions on Google Cloud in one place.

* `vertex_ai_model_version_aliases allows you to set aliases to help you keep track of your model version (<a href="https://cloud.google.com/vertex-ai/docs/model-registry/model-alias" target="_blank">documentation</a>).

In [ ]:
# !pip install google-cloud-bigquery-datatransfer -q

In [37]:
from google.cloud import bigquery_datatransfer

transfer_client = bigquery_datatransfer.DataTransferServiceClient()

destination_project_id = PROJECT_ID
destination_dataset_id = BQ_DATASET_NAME
source_project_id = "bqmlpublic"
source_dataset_id = "demo_ga4churnprediction"
transfer_config = bigquery_datatransfer.TransferConfig(
    destination_dataset_id=destination_dataset_id,
    display_name="Copy of bqmlpublic demo_ga4churnprediction",
    data_source_id="cross_region_copy",
    params={
        "source_project_id": source_project_id,
        "source_dataset_id": source_dataset_id,
    },
    # schedule="every 24 hours",
)
transfer_config = transfer_client.create_transfer_config(
    parent=transfer_client.common_project_path(destination_project_id),
    transfer_config=transfer_config,
)
print(f"Created transfer config: {transfer_config.name}")

Created transfer config: projects/55590906972/locations/us-central1/transferConfigs/641043c2-0000-2de8-8b7f-582429c88618


In [34]:
bq_client = bigquery.Client(project=PROJECT_ID, location=REGION)

In [39]:
# this cell may take ~1 min to run

BQML_MODEL_NAME = f"bqml_model_churn_{UUID}"

sql_train_model_bqml = f"""
CREATE OR REPLACE MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME}    
OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  input_label_cols=["churned"],
  model_registry="vertex_ai",
  vertex_ai_model_version_aliases=['logistic_reg', 'experimental']
) AS

SELECT
  * EXCEPT(user_first_engagement, user_pseudo_id)
FROM
  {PROJECT_ID}.ga4_churnprediction_0obvksai.training_data
"""

print(sql_train_model_bqml)

run_bq_query(sql_train_model_bqml)


CREATE OR REPLACE MODEL ga4_churnprediction_0obvksai.bqml_model_churn_0obvksai    
OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  input_label_cols=["churned"],
  model_registry="vertex_ai",
  vertex_ai_model_version_aliases=['logistic_reg', 'experimental']
) AS

SELECT
  * EXCEPT(user_first_engagement, user_pseudo_id)
FROM
  mwpmltr.ga4_churnprediction_0obvksai.training_data

Finished job_id: 356619dd-8f9a-42e4-abee-69ffb1f6e515


""


### Model evaluation

With the model created, you can now evaluate the logistic regression model. Behind the scenes, BigQuery ML automatically <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method" target="_blank">split the data</a>, which makes it easier to quickly train and evaluate models.

In [ ]:
sql_evaluate_model = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME})
"""

print(sql_evaluate_model)

run_bq_query(sql_evaluate_model)


SELECT
  *
FROM
  ML.EVALUATE(MODEL ga4_churnprediction_0obvksai.bqml_model_churn_0obvksai)

Finished job_id: 51ecbe30-fef7-4f70-a44e-eb4bc4a30a84


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.571924,0.995775,0.571083,0.726552,0.671342,0.602277


These metrics help you understand the performance of the model. 

There are various metrics for logistic regression and other model types (full list of metrics can be found in the <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate#mlevaluate_output" target="_blank">documentation</a>).

### Batch prediction (with Explainable AI)

Make a batch prediction in BigQuery ML on the original training data to check the probability of churn for each of the users, as seen in the `probability` column, with the predicted label under the `predicted_churn` column.

<a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict" target="_blank">ML.EXPLAIN_PREDICT</a> has built-in <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview" target="_blank">Explainable AI</a>. This allows you to see the top contributing features to each prediction and interpret how it was computed.

In [44]:
sql_explain_predict = f"""
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME},
    (SELECT * FROM {PROJECT_ID}.ga4_churnprediction_0obvksai.training_data LIMIT 100)
    )
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)


SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL ga4_churnprediction_0obvksai.bqml_model_churn_0obvksai,
    (SELECT * FROM mwpmltr.ga4_churnprediction_0obvksai.training_data LIMIT 100)
    )

Finished job_id: 65435635-4870-4401-907f-5e09d95567ac


,predicted_churned,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,user_pseudo_id,country,operating_system,language,...,cnt_add_to_cart,cnt_begin_checkout,cnt_add_shipping_info,cnt_add_payment_info,cnt_purchase,user_first_engagement,month,julianday,dayofweek,churned
0,1,0.595976,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.388725,0.0,5298013.7728745800,United States,Web,en-us,...,0,0,0,0,0,2021-01-01 13:59:53.752059+00:00,1,1,6,0
1,1,0.622918,"[{'feature': 'month', 'attribution': 0.0808542...",0.069157,0.501954,0.0,22288916.8827658539,United Kingdom,Web,None,...,3,0,0,0,0,2021-01-01 11:21:20.741223+00:00,1,1,6,0
2,1,0.583805,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.338415,0.0,3573143.3095259411,United States,Web,en-us,...,0,0,0,0,0,2021-01-01 06:23:08.482574+00:00,1,1,6,1
3,1,0.582265,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.332077,0.0,5115768.1503494300,United States,Web,en-us,...,0,0,0,0,0,2021-01-01 20:07:22.264757+00:00,1,1,6,0
4,1,0.585443,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.345158,0.0,2914301.0348640426,Austria,Windows,en-us,...,0,0,0,0,0,2021-01-01 23:30:06.990337+00:00,1,1,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0.600546,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.407739,0.0,13623777.5841147931,United States,Web,en-us,...,0,0,0,0,0,2021-01-01 01:16:07.770257+00:00,1,1,6,0
96,1,0.578926,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.318367,0.0,6351413.2459505432,Morocco,Web,en-us,...,0,0,0,0,0,2021-01-01 01:00:01.910471+00:00,1,1,6,1
97,1,0.594815,"[{'feature': 'language', 'attribution': 0.0809...",0.069157,0.383905,0.0,8284309.3576851305,United Kingdom,Web,en-gb,...,0,0,0,0,0,2021-01-01 19:10:08.402348+00:00,1,1,6,0
98,1,0.568162,"[{'feature': 'month', 'attribution': 0.0808542...",0.069157,0.274355,0.0,44673777.8459608727,United States,Web,None,...,0,0,0,0,0,2021-01-01 10:13:22.966736+00:00,1,1,6,0


Since the `top_feature_attributions` is a nested column, you can unnest the array (<a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays" target="_blank">documentation</a>) into separate rows for each of the features. In other words, since ML.EXPLAIN_PREDICT provides the top 5 most important features, using `UNNEST` results in 5 rows per prediction:

In [45]:
sql_explain_predict = f"""
SELECT
  tfa.*,
  predicted_churned,
  probability,
  baseline_prediction_value,
  prediction_value,
  approximation_error,
  user_pseudo_id
FROM
  ML.EXPLAIN_PREDICT(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME},
    (SELECT * FROM {PROJECT_ID}.ga4_churnprediction_0obvksai.training_data LIMIT 100)
    ),
  UNNEST(top_feature_attributions) as tfa
WHERE
  user_pseudo_id = "7666337.2408476627"
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)


SELECT
  tfa.*,
  predicted_churned,
  probability,
  baseline_prediction_value,
  prediction_value,
  approximation_error,
  user_pseudo_id
FROM
  ML.EXPLAIN_PREDICT(MODEL ga4_churnprediction_0obvksai.bqml_model_churn_0obvksai,
    (SELECT * FROM mwpmltr.ga4_churnprediction_0obvksai.training_data LIMIT 100)
    ),
  UNNEST(top_feature_attributions) as tfa
WHERE
  user_pseudo_id = "7666337.2408476627"

Finished job_id: ab158a5b-da29-4622-a588-a1422e274572


,feature,attribution,predicted_churned,probability,baseline_prediction_value,prediction_value,approximation_error,user_pseudo_id
0,language,0.097598,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627
1,month,0.080854,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627
2,julianday,0.080515,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627
3,operating_system,0.075108,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627
4,country,0.074942,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627


### Inspect the model on Vertex AI Model Registry

When the model was trained in BigQuery ML, the line `model_registry="vertex_ai"` registered the model to Vertex AI Model Registry automatically upon completion.

You can view the model on the <a href="https://console.cloud.google.com/vertex-ai/models" target="_blank">Vertex AI Model Registry page</a>, or use the code below to check that it was successfully registered:

In [46]:
model = vertex_ai.Model(model_name=BQML_MODEL_NAME)

print(model.gca_resource)

name: "projects/55590906972/locations/us-central1/models/bqml_model_churn_0obvksai"
display_name: "bqml_model_churn_0obvksai"
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1678928262
  nanos: 157133000
}
update_time {
  seconds: 1678928266
  nanos: 498782000
}
etag: "AMEw9yMt0wt5Lrd4YBY4FM3MtBD6OFwC9s3XQ_FZK3rQGWNsaUErb8fnjaLd-RwryFUx"
version_id: "1"
version_aliases: "logistic_reg"
version_aliases: "experimental"
version_aliases: "default"
version_create_time {
  seconds: 1678928262
  nanos: 157133000
}
version_update_time {
  seconds: 1678928280
  nanos: 948554000
}
model_source_info {
  source_type: BQ

### Deploy the model to an endpoint

While BigQuery ML supports batch prediction with <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict" target="_blank">ML.PREDICT</a> and <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict" target="_blank">ML.EXPLAIN_PREDICT</a>, BigQuery ML is not suitable for real-time predictions where you need low latency predictions with potentially high frequency of requests.

In other words, deploying the BigQuery ML model to an endpoint enables you to do online predictions.

#### Create a Vertex AI endpoint

To deploy your model to an endpoint, you will first need to create an endpoint before you deploy the model to it.

In [47]:
ENDPOINT_NAME = f"{BQML_MODEL_NAME}-endpoint"

endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

Creating Endpoint
Create Endpoint backing LRO: projects/55590906972/locations/us-central1/endpoints/1791179208437792768/operations/9133064542660591616
Endpoint created. Resource name: projects/55590906972/locations/us-central1/endpoints/1791179208437792768
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/55590906972/locations/us-central1/endpoints/1791179208437792768')
bqml_model_churn_0obvksai-endpoint
projects/55590906972/locations/us-central1/endpoints/1791179208437792768


#### List endpoints

List the endpoints to make sure it has successfully been created. (You can also view your endpoints on the <a href="https://console.cloud.google.com/vertex-ai/endpoints" target="_blank">Vertex AI Endpoints page</a>).

In [48]:
endpoint.list()

 resource name: projects/55590906972/locations/us-central1/endpoints/1791179208437792768,
 resource name: projects/55590906972/locations/us-central1/endpoints/4825479457378664448,
 resource name: projects/55590906972/locations/us-central1/endpoints/7174669613005799424,
 resource name: projects/55590906972/locations/us-central1/endpoints/4513323708206546944,
 resource name: projects/55590906972/locations/us-central1/endpoints/3582204485247696896,
 resource name: projects/55590906972/locations/us-central1/endpoints/6778634320773906432,
 resource name: projects/55590906972/locations/us-central1/endpoints/3602110043756953600,
 resource name: projects/55590906972/locations/us-central1/endpoints/549654658818244608,
 resource name: projects/55590906972/locations/us-central1/endpoints/4925803296343457792,
 resource name: projects/55590906972/locations/us-central1/endpoints/8684492591988736000,
 resource name: projects/55590906972/locations/us-central1/endpoints/3874177598980358144,
 resource n

#### Deploy model to Vertex endpoint

With the new endpoint, you can now deploy your model.

In [49]:
# deploying the model to the endpoint may take 10-15 minutes
model.deploy(endpoint=endpoint)

Deploying model to Endpoint : projects/55590906972/locations/us-central1/endpoints/1791179208437792768
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/55590906972/locations/us-central1/endpoints/1791179208437792768/operations/7527531275503009792
Endpoint model deployed. Resource name: projects/55590906972/locations/us-central1/endpoints/1791179208437792768


resource name: projects/55590906972/locations/us-central1/endpoints/1791179208437792768

You can also check on the status of your model by visiting the <a href="https://console.cloud.google.com/vertex-ai/endpoints" target="_blank">Vertex AI Endpoints page</a>.

### Make online predictions to the endpoint

Using a sample of the training data, you can test the endpoint to make online predictions.

In [50]:
df_sample_requests_list = [
    {
        "country": "Turkey",
        "operating_system": "Web",
        "language": "None",
        "cnt_user_engagement": 28,
        "cnt_page_view": 37,
        "cnt_view_item": 6,
        "cnt_view_promotion": 15,
        "cnt_select_promotion": 4,
        "cnt_add_to_cart": 0,
        "cnt_begin_checkout": 0,
        "cnt_add_shipping_info": 0,
        "cnt_add_payment_info": 0,
        "cnt_purchase": 0,
        "month": 1,
        "julianday": 1,
        "dayofweek": 6,
    },
    {
        "country": "Macao",
        "operating_system": "Web",
        "language": "None",
        "cnt_user_engagement": 2,
        "cnt_page_view": 4,
        "cnt_view_item": 0,
        "cnt_view_promotion": 0,
        "cnt_select_promotion": 0,
        "cnt_add_to_cart": 0,
        "cnt_begin_checkout": 0,
        "cnt_add_shipping_info": 0,
        "cnt_add_payment_info": 0,
        "cnt_purchase": 0,
        "month": 1,
        "julianday": 16,
        "dayofweek": 7,
    },
]

In [51]:
prediction = endpoint.predict(df_sample_requests_list)
print(prediction)

Prediction(predictions=[{'churned_probs': [0.754157827593266, 0.245842172406734], 'predicted_churned': ['1'], 'churned_values': ['1', '0']}, {'churned_values': ['1', '0'], 'predicted_churned': ['0'], 'churned_probs': [0.4827901063200341, 0.517209893679966]}], deployed_model_id='1564714997488025600', model_version_id='1', model_resource_name='projects/55590906972/locations/us-central1/models/bqml_model_churn_0obvksai', explanations=None)


You can then extract the predictions from the prediction response

In [52]:
prediction.predictions

[{'churned_probs': [0.754157827593266, 0.245842172406734],
  'predicted_churned': ['1'],
  'churned_values': ['1', '0']},
 {'churned_values': ['1', '0'],
  'predicted_churned': ['0'],
  'churned_probs': [0.4827901063200341, 0.517209893679966]}]

## Cleaning up

To clean up all Google Cloud resources used in this project, you can <a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">delete the Google Cloud
project</a> you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [53]:
# Undeploy model from endpoint and delete endpoint
endpoint.undeploy_all()
endpoint.delete()

# Delete BigQuery dataset, including the BigQuery ML model
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME

Undeploying Endpoint model: projects/55590906972/locations/us-central1/endpoints/1791179208437792768
Undeploy Endpoint model backing LRO: projects/55590906972/locations/us-central1/endpoints/1791179208437792768/operations/3593637000994881536
Endpoint model undeployed. Resource name: projects/55590906972/locations/us-central1/endpoints/1791179208437792768
Deleting Endpoint : projects/55590906972/locations/us-central1/endpoints/1791179208437792768
Delete Endpoint  backing LRO: projects/55590906972/locations/us-central1/operations/536115066475380736
Endpoint deleted. . Resource name: projects/55590906972/locations/us-central1/endpoints/1791179208437792768
